In [1]:
import joblib
import yaml

import pandas as pd

from sklearn.preprocessing import OneHotEncoder

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [2]:
PATH_CONFIG = '../config/config.yaml'
config = yaml.safe_load(open(PATH_CONFIG))

Load train features and label data

In [4]:
X_train_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['X_train']
y_train_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['y_train']

X_train = joblib.load(X_train_path)
y_train = joblib.load(y_train_path)

**PRE-PROCESSING**

In [5]:
pd.set_option('display.max_columns', 500)
X_train.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year
889,191,38,753005,2005-11-20 00:00:00,IL,100/300,2000,1253.44,0,608525,FEMALE,Masters,craft-repair,sleeping,not-in-family,0,0,2015-02-07 00:00:00,Multi-vehicle Collision,Side Collision,Major Damage,Ambulance,VA,Hillsdale,7201 Washington Ave,19,3,NO,2,0,NO,56320,10240,5120,40960,Volkswagen,Jetta,2007
692,137,35,153027,2010-03-11 00:00:00,IN,250/500,500,1667.83,0,460586,MALE,JD,prof-specialty,paintball,husband,48500,-67400,2015-02-04 00:00:00,Parked Car,?,Minor Damage,Police,WV,Northbrook,4447 Francis Hwy,4,1,YES,1,1,NO,6600,1200,1200,4200,Jeep,Grand Cherokee,2005
621,87,27,326289,2004-01-03 00:00:00,OH,100/300,500,1048.39,0,620962,FEMALE,Masters,transport-moving,polo,own-child,0,0,2015-02-13 00:00:00,Single Vehicle Collision,Side Collision,Minor Damage,Police,NY,Riverwood,6608 Apache Lane,2,1,?,2,1,YES,34650,6300,3150,25200,Ford,F150,1996
355,256,43,883980,2014-12-13 00:00:00,OH,100/300,500,1203.17,0,455810,FEMALE,MD,prof-specialty,golf,unmarried,56700,-65600,2015-02-06 00:00:00,Single Vehicle Collision,Rear Collision,Total Loss,Fire,WV,Hillsdale,8834 Elm Drive,11,1,NO,0,0,?,63250,11500,5750,46000,Nissan,Ultima,1997
362,150,30,354481,2004-11-17 00:00:00,IN,100/300,1000,1342.02,0,608425,MALE,MD,prof-specialty,polo,own-child,0,0,2015-02-28 00:00:00,Parked Car,?,Trivial Damage,NaN,VA,Arlington,6317 Best St,8,1,YES,0,2,NO,4500,450,450,3600,Saab,93,1999


In [6]:
y_train.head()

889    N
692    N
621    N
355    N
362    N
Name: fraud_reported, dtype: object

Sanity check shape of train features and label data

In [7]:
print(f'Shape of train data features: {X_train.shape}')
print(f'Shape of train label : {y_train.shape}')

Shape of train data features: (700, 38)
Shape of train label : (700,)


Doing features selection, fixing data type, and handling missing values

In [8]:
# defined function to remove irrelevant features, fixing data type, and handling missing values
def preprocess_input(X,
                     datetime_features,
                     num_int_features,
                     num_float_features,
                     cat_features,
                     drop_features):
    
    # remove irrelevant features
    X = X.drop(drop_features, axis = 1)

    # fixing data type
    X[datetime_features] = pd.to_datetime(X[datetime_features].stack()).unstack()
    X[cat_features] = X[cat_features].astype(object)
    X[num_int_features] = X[num_int_features].astype(int)
    X[num_float_features] = X[num_float_features].astype(float)

    # handling missing values
    X[cat_features] = X[cat_features].replace('?', None)
    X[cat_features] = X[cat_features].fillna('UNKNOWN')
    
    return X

In [9]:
datetime_features = config['preprocess']['datetime_features']
cat_features = config['preprocess']['cat_features']
num_int_features = config['preprocess']['num_int_features']
num_float_features = config['preprocess']['num_float_features']

drop_features = config['preprocess']['drop_features']

X_train_preprocess = preprocess_input(X_train,
                                      datetime_features,
                                      num_int_features, 
                                      num_float_features,
                                      cat_features,
                                      drop_features)

Sanity check data type and missing value

In [10]:
X_train_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700 entries, 889 to 395
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   age                          700 non-null    int64  
 1   policy_state                 700 non-null    object 
 2   policy_csl                   700 non-null    object 
 3   policy_deductable            700 non-null    float64
 4   policy_annual_premium        700 non-null    float64
 5   umbrella_limit               700 non-null    float64
 6   insured_sex                  700 non-null    object 
 7   insured_education_level      700 non-null    object 
 8   insured_occupation           700 non-null    object 
 9   insured_hobbies              700 non-null    object 
 10  insured_relationship         700 non-null    object 
 11  capital-gains                700 non-null    float64
 12  capital-loss                 700 non-null    float64
 13  incident_type          

There's no missing value and the data type has suit the values

Sanity check duplicated data

In [11]:
X_train_preprocess.duplicated().sum()

0

**FEATURE ENGINEERING**

One hot encoding for categorical features

In [12]:
def ohe_input(X,
              X_train,
              cat_features):
    ohe = OneHotEncoder(handle_unknown = 'ignore')
    ohe.fit(X_train[cat_features])
    ohe_values = ohe.transform(X[cat_features]).toarray()
    X_index = X.index
    X_columns = ohe.get_feature_names_out()
    X_ohe = pd.DataFrame(ohe_values, index=X_index, columns=X_columns)
    
    return X_ohe

In [13]:
X_train_ohe = ohe_input(X_train_preprocess, X_train_preprocess, cat_features)
X_train_ohe.head()

,policy_state_IL,policy_state_IN,policy_state_OH,policy_csl_100/300,policy_csl_250/500,policy_csl_500/1000,insured_sex_FEMALE,insured_sex_MALE,insured_education_level_Associate,insured_education_level_College,insured_education_level_High School,insured_education_level_JD,insured_education_level_MD,insured_education_level_Masters,insured_education_level_PhD,insured_occupation_adm-clerical,insured_occupation_armed-forces,insured_occupation_craft-repair,insured_occupation_exec-managerial,insured_occupation_farming-fishing,insured_occupation_handlers-cleaners,insured_occupation_machine-op-inspct,insured_occupation_other-service,insured_occupation_priv-house-serv,insured_occupation_prof-specialty,insured_occupation_protective-serv,insured_occupation_sales,insured_occupation_tech-support,insured_occupation_transport-moving,insured_hobbies_base-jumping,insured_hobbies_basketball,insured_hobbies_board-games,insured_hobbies_bungie-jumping,insured_hobbies_camping,insured_hobbies_chess,insured_hobbies_cross-fit,insured_hobbies_dancing,insured_hobbies_exercise,insured_hobbies_golf,insured_hobbies_hiking,insured_hobbies_kayaking,insured_hobbies_movies,insured_hobbies_paintball,insured_hobbies_polo,insured_hobbies_reading,insured_hobbies_skydiving,insured_hobbies_sleeping,insured_hobbies_video-games,insured_hobbies_yachting,insured_relationship_husband,insured_relationship_not-in-family,insured_relationship_other-relative,insured_relationship_own-child,insured_relationship_unmarried,insured_relationship_wife,incident_type_Multi-vehicle Collision,incident_type_Parked Car,incident_type_Single Vehicle Collision,incident_type_Vehicle Theft,collision_type_Front Collision,collision_type_Rear Collision,collision_type_Side Collision,collision_type_UNKNOWN,incident_severity_Major Damage,incident_severity_Minor Damage,incident_severity_Total Loss,incident_severity_Trivial Damage,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_Other,authorities_contacted_Police,authorities_contacted_UNKNOWN,incident_state_NC,incident_state_NY,incident_state_OH,incident_state_PA,incident_state_SC,incident_state_VA,incident_state_WV,property_damage_NO,property_damage_UNKNOWN,property_damage_YES,police_report_available_NO,police_report_available_UNKNOWN,police_report_available_YES,auto_make_Accura,auto_make_Audi,auto_make_BMW,auto_make_Chevrolet,auto_make_Dodge,auto_make_Ford,auto_make_Honda,auto_make_Jeep,auto_make_Mercedes,auto_make_Nissan,auto_make_Saab,auto_make_Suburu,auto_make_Toyota,auto_make_Volkswagen,auto_model_3 Series,auto_model_92x,auto_model_93,auto_model_95,auto_model_A3,auto_model_A5,auto_model_Accord,auto_model_C300,auto_model_CRV,auto_model_Camry,auto_model_Civic,auto_model_Corolla,auto_model_E400,auto_model_Escape,auto_model_F150,auto_model_Forrestor,auto_model_Fusion,auto_model_Grand Cherokee,auto_model_Highlander,auto_model_Impreza,auto_model_Jetta,auto_model_Legacy,auto_model_M5,auto_model_MDX,auto_model_ML350,auto_model_Malibu,auto_model_Maxima,auto_model_Neon,auto_model_Passat,auto_model_Pathfinder,auto_model_RAM,auto_model_RSX,auto_model_Silverado,auto_model_TL,auto_model_Tahoe,auto_model_Ultima,auto_model_Wrangler,auto_model_X5,auto_model_X6,auto_year_1995,auto_year_1996,auto_year_1997,auto_year_1998,auto_year_1999,auto_year_2000,auto_year_2001,auto_year_2002,auto_year_2003,auto_year_2004,auto_year_2005,auto_year_2006,auto_year_2007,auto_year_2008,auto_year_2009,auto_year_2010,auto_year_2011,auto_year_2012,auto_year_2013,auto_year_2014,auto_year_2015
889,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Concat numerical and categorical data to be one table

In [14]:
num_features = config['preprocess']['num_features']

def clean_input(X_preprocess, X_ohe):
    X_clean = pd.concat([X_preprocess[num_features], X_ohe], axis=1)

    return X_clean

In [15]:
X_train_clean = clean_input(X_train_preprocess, X_train_ohe)
X_train_clean.head()

,age,number_of_vehicles_involved,bodily_injuries,witnesses,auto_year,policy_deductable,policy_annual_premium,umbrella_limit,capital-gains,capital-loss,total_claim_amount,injury_claim,property_claim,vehicle_claim,policy_state_IL,policy_state_IN,policy_state_OH,policy_csl_100/300,policy_csl_250/500,policy_csl_500/1000,insured_sex_FEMALE,insured_sex_MALE,insured_education_level_Associate,insured_education_level_College,insured_education_level_High School,insured_education_level_JD,insured_education_level_MD,insured_education_level_Masters,insured_education_level_PhD,insured_occupation_adm-clerical,insured_occupation_armed-forces,insured_occupation_craft-repair,insured_occupation_exec-managerial,insured_occupation_farming-fishing,insured_occupation_handlers-cleaners,insured_occupation_machine-op-inspct,insured_occupation_other-service,insured_occupation_priv-house-serv,insured_occupation_prof-specialty,insured_occupation_protective-serv,insured_occupation_sales,insured_occupation_tech-support,insured_occupation_transport-moving,insured_hobbies_base-jumping,insured_hobbies_basketball,insured_hobbies_board-games,insured_hobbies_bungie-jumping,insured_hobbies_camping,insured_hobbies_chess,insured_hobbies_cross-fit,insured_hobbies_dancing,insured_hobbies_exercise,insured_hobbies_golf,insured_hobbies_hiking,insured_hobbies_kayaking,insured_hobbies_movies,insured_hobbies_paintball,insured_hobbies_polo,insured_hobbies_reading,insured_hobbies_skydiving,insured_hobbies_sleeping,insured_hobbies_video-games,insured_hobbies_yachting,insured_relationship_husband,insured_relationship_not-in-family,insured_relationship_other-relative,insured_relationship_own-child,insured_relationship_unmarried,insured_relationship_wife,incident_type_Multi-vehicle Collision,incident_type_Parked Car,incident_type_Single Vehicle Collision,incident_type_Vehicle Theft,collision_type_Front Collision,collision_type_Rear Collision,collision_type_Side Collision,collision_type_UNKNOWN,incident_severity_Major Damage,incident_severity_Minor Damage,incident_severity_Total Loss,incident_severity_Trivial Damage,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_Other,authorities_contacted_Police,authorities_contacted_UNKNOWN,incident_state_NC,incident_state_NY,incident_state_OH,incident_state_PA,incident_state_SC,incident_state_VA,incident_state_WV,property_damage_NO,property_damage_UNKNOWN,property_damage_YES,police_report_available_NO,police_report_available_UNKNOWN,police_report_available_YES,auto_make_Accura,auto_make_Audi,auto_make_BMW,auto_make_Chevrolet,auto_make_Dodge,auto_make_Ford,auto_make_Honda,auto_make_Jeep,auto_make_Mercedes,auto_make_Nissan,auto_make_Saab,auto_make_Suburu,auto_make_Toyota,auto_make_Volkswagen,auto_model_3 Series,auto_model_92x,auto_model_93,auto_model_95,auto_model_A3,auto_model_A5,auto_model_Accord,auto_model_C300,auto_model_CRV,auto_model_Camry,auto_model_Civic,auto_model_Corolla,auto_model_E400,auto_model_Escape,auto_model_F150,auto_model_Forrestor,auto_model_Fusion,auto_model_Grand Cherokee,auto_model_Highlander,auto_model_Impreza,auto_model_Jetta,auto_model_Legacy,auto_model_M5,auto_model_MDX,auto_model_ML350,auto_model_Malibu,auto_model_Maxima,auto_model_Neon,auto_model_Passat,auto_model_Pathfinder,auto_model_RAM,auto_model_RSX,auto_model_Silverado,auto_model_TL,auto_model_Tahoe,auto_model_Ultima,auto_model_Wrangler,auto_model_X5,auto_model_X6,auto_year_1995,auto_year_1996,auto_year_1997,auto_year_1998,auto_year_1999,auto_year_2000,auto_year_2001,auto_year_2002,auto_year_2003,auto_year_2004,auto_year_2005,auto_year_2006,auto_year_2007,auto_year_2008,auto_year_2009,auto_year_2010,auto_year_2011,auto_year_2012,auto_year_2013,auto_year_2014,auto_year_2015
889,38,3,2,0,2007,2000.0,1253.44,0.0,0.0,0.0,56320.0,10240.0,5120.0,40960.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Encoding label data, with 0 represent Not Fraud and 1 represent Fraud

In [16]:
def ohe_output(y):
    y_ohe = y.replace({'Y': 1, 'N': 0})

    return y_ohe

In [17]:
y_train_clean = ohe_output(y_train)
y_train_clean.head()

889    0
692    0
621    0
355    0
362    0
Name: fraud_reported, dtype: int64

Balancing label using two method i.e., RandomUnderSampler and RandomOverSampler

In [18]:
def resampling(X, 
               y, 
               methods = None):
    
    if methods == None:
        X_sam, y_sam = X, y

    else:
        sampler = methods(random_state = config['feature_eng']['sampler_random_state'])
        X_sam, y_sam = sampler.fit_resample(X, y)

    return X_sam, y_sam

In [19]:
X_train_clean_rus, y_train_clean_rus = resampling(X_train_clean, y_train_clean, methods = RandomUnderSampler)

In [20]:
X_train_clean_ros, y_train_clean_ros = resampling(X_train_clean, y_train_clean, methods = RandomOverSampler)

Sanity check imbalance label

In [21]:
y_train_clean_rus.value_counts(normalize = True)

fraud_reported
0    0.5
1    0.5
Name: proportion, dtype: float64

In [22]:
y_train_clean_ros.value_counts(normalize = True)

fraud_reported
0    0.5
1    0.5
Name: proportion, dtype: float64

In [23]:
# store train clean dataset
X_train_clean_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['X_train_clean']
y_train_clean_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['y_train_clean']
joblib.dump(X_train_clean, X_train_clean_path)
joblib.dump(y_train_clean, y_train_clean_path)

X_train_clean_rus_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['X_train_clean_rus']
y_train_clean_rus_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['y_train_clean_rus']
joblib.dump(X_train_clean_rus, X_train_clean_rus_path)
joblib.dump(y_train_clean_rus, y_train_clean_rus_path)

X_train_clean_ros_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['X_train_clean_ros']
y_train_clean_ros_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['y_train_clean_ros']
joblib.dump(X_train_clean_ros, X_train_clean_ros_path)
joblib.dump(y_train_clean_ros, y_train_clean_ros_path)

['../data/processed/y_train_clean_ros.pkl']

Pre-processing and Feature Engineering Test Data

In [24]:
X_test_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['X_test']
y_test_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['y_test']

X_test = joblib.load(X_test_path)
y_test = joblib.load(y_test_path)

In [25]:
X_test_preprocess =  preprocess_input(X_test,
                                      datetime_features,
                                      num_int_features,
                                      num_float_features,
                                      cat_features,
                                      drop_features)

X_test_ohe = ohe_input(X_test_preprocess, 
                        X_train_preprocess, 
                        cat_features)

X_test_clean = clean_input(X_test_preprocess, X_test_ohe)

In [26]:
y_test_clean = ohe_output(y = y_test)

In [27]:
X_test_clean.head()

,age,number_of_vehicles_involved,bodily_injuries,witnesses,auto_year,policy_deductable,policy_annual_premium,umbrella_limit,capital-gains,capital-loss,total_claim_amount,injury_claim,property_claim,vehicle_claim,policy_state_IL,policy_state_IN,policy_state_OH,policy_csl_100/300,policy_csl_250/500,policy_csl_500/1000,insured_sex_FEMALE,insured_sex_MALE,insured_education_level_Associate,insured_education_level_College,insured_education_level_High School,insured_education_level_JD,insured_education_level_MD,insured_education_level_Masters,insured_education_level_PhD,insured_occupation_adm-clerical,insured_occupation_armed-forces,insured_occupation_craft-repair,insured_occupation_exec-managerial,insured_occupation_farming-fishing,insured_occupation_handlers-cleaners,insured_occupation_machine-op-inspct,insured_occupation_other-service,insured_occupation_priv-house-serv,insured_occupation_prof-specialty,insured_occupation_protective-serv,insured_occupation_sales,insured_occupation_tech-support,insured_occupation_transport-moving,insured_hobbies_base-jumping,insured_hobbies_basketball,insured_hobbies_board-games,insured_hobbies_bungie-jumping,insured_hobbies_camping,insured_hobbies_chess,insured_hobbies_cross-fit,insured_hobbies_dancing,insured_hobbies_exercise,insured_hobbies_golf,insured_hobbies_hiking,insured_hobbies_kayaking,insured_hobbies_movies,insured_hobbies_paintball,insured_hobbies_polo,insured_hobbies_reading,insured_hobbies_skydiving,insured_hobbies_sleeping,insured_hobbies_video-games,insured_hobbies_yachting,insured_relationship_husband,insured_relationship_not-in-family,insured_relationship_other-relative,insured_relationship_own-child,insured_relationship_unmarried,insured_relationship_wife,incident_type_Multi-vehicle Collision,incident_type_Parked Car,incident_type_Single Vehicle Collision,incident_type_Vehicle Theft,collision_type_Front Collision,collision_type_Rear Collision,collision_type_Side Collision,collision_type_UNKNOWN,incident_severity_Major Damage,incident_severity_Minor Damage,incident_severity_Total Loss,incident_severity_Trivial Damage,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_Other,authorities_contacted_Police,authorities_contacted_UNKNOWN,incident_state_NC,incident_state_NY,incident_state_OH,incident_state_PA,incident_state_SC,incident_state_VA,incident_state_WV,property_damage_NO,property_damage_UNKNOWN,property_damage_YES,police_report_available_NO,police_report_available_UNKNOWN,police_report_available_YES,auto_make_Accura,auto_make_Audi,auto_make_BMW,auto_make_Chevrolet,auto_make_Dodge,auto_make_Ford,auto_make_Honda,auto_make_Jeep,auto_make_Mercedes,auto_make_Nissan,auto_make_Saab,auto_make_Suburu,auto_make_Toyota,auto_make_Volkswagen,auto_model_3 Series,auto_model_92x,auto_model_93,auto_model_95,auto_model_A3,auto_model_A5,auto_model_Accord,auto_model_C300,auto_model_CRV,auto_model_Camry,auto_model_Civic,auto_model_Corolla,auto_model_E400,auto_model_Escape,auto_model_F150,auto_model_Forrestor,auto_model_Fusion,auto_model_Grand Cherokee,auto_model_Highlander,auto_model_Impreza,auto_model_Jetta,auto_model_Legacy,auto_model_M5,auto_model_MDX,auto_model_ML350,auto_model_Malibu,auto_model_Maxima,auto_model_Neon,auto_model_Passat,auto_model_Pathfinder,auto_model_RAM,auto_model_RSX,auto_model_Silverado,auto_model_TL,auto_model_Tahoe,auto_model_Ultima,auto_model_Wrangler,auto_model_X5,auto_model_X6,auto_year_1995,auto_year_1996,auto_year_1997,auto_year_1998,auto_year_1999,auto_year_2000,auto_year_2001,auto_year_2002,auto_year_2003,auto_year_2004,auto_year_2005,auto_year_2006,auto_year_2007,auto_year_2008,auto_year_2009,auto_year_2010,auto_year_2011,auto_year_2012,auto_year_2013,auto_year_2014,auto_year_2015
55,26,1,1,2,1995,1000.0,1276.57,0.0,71500.0,0.0,78900.0,15780.0,7890.0,55230.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1

In [28]:
y_test_clean.head()

55     0
336    0
892    0
454    0
79     1
Name: fraud_reported, dtype: int64

In [29]:
# store test clean dataset
X_test_clean_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['X_test_clean']
y_test_clean_path = '../' + config['train_test_data']['directory'] + config['train_test_data']['y_test_clean']

joblib.dump(X_test_clean, X_test_clean_path)
joblib.dump(y_test_clean, y_test_clean_path)

['../data/processed/y_test_clean.pkl']